In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd

In [ ]:
books = pd.read_csv(f"{path}/books.csv")

In [ ]:
books

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Check missing values
ax = plt.axes()
sns.heatmap(books.isna().transpose(), cbar=False, ax=ax)

plt.xlabel("Column")
plt.ylabel("Missing Values")

plt.show()

In [ ]:
# Create missing description, and age cols
import numpy as np

books["missing_description"] = np.where(books["description"].isna(), 1, 0)
books['age_of_book'] = 2024 - books['published_year']

In [ ]:
# Create correlation matrix to know relation between missing_description and other attributes
columns_of_interest = ['num_pages', 'age_of_book', 'missing_description', 'average_rating']

correlation_matrix = books[columns_of_interest].corr(method="spearman")

sns.set_theme(style="white")
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt='.2f',cmap="coolwarm", cbar_kws={"label": "Spearman Correlation"})
heatmap.set_title("Correlation Heatmap")
plt.show()

In [ ]:
# Recreate DF to filter null values in 4 cols
book_missing = books[~(books['description'].isna()) &
        ~(books['num_pages'].isna()) &
        ~(books['published_year'].isna()) &
        ~(books['average_rating'].isna())
]

In [ ]:
book_missing

In [ ]:
# Check the categories cols (for recommendation)
book_missing['categories'].value_counts().reset_index().sort_values('count', ascending=False)

In [ ]:
# Create new cols for description word count
book_missing['words_in_description'] = book_missing['description'].str.split().str.len()

In [ ]:
book_missing

In [ ]:
# If the description is too short (< 25 words) then it doesn't contain enough infor
book_missing.loc[book_missing['words_in_description'].between(1, 24), 'description']

In [ ]:
# Recreate DF to filter short description
book_missing_25_words = book_missing[book_missing['words_in_description'] >= 25]

In [ ]:
book_missing_25_words

In [ ]:
# Add new cols to merge title and subtitle (if subtitle not nan)
book_missing_25_words['title_and_subtitle'] = np.where(book_missing_25_words['subtitle'].isna(), book_missing_25_words['title'], book_missing_25_words[['title'], ['subtitle']].astype(str).agg(": ".join, axis=1))

In [ ]:
book_missing_25_words

In [ ]:
# Merge 'isbn13' and 'description' to create distinct description values
book_missing_25_words['tagged_description'] = book_missing_25_words[['isbn13', 'description']].astype(str).agg(" ".join, axis=1)

In [ ]:
book_missing_25_words

In [ ]:
# Drop extra cols and save to csv file
(
    book_missing_25_words
    .drop(['subtitle', 'missing_description', 'age_of_book', 'words_in_description'], axis=1)
    .to_csv(f"books_cleaned.csv", index=False)
)